# 📊 MemoLib Production Deployment Monitoring

**Date**: 6 février 2026  
**Status**: 🚀 Deployment in Progress  
**Objective**: Real-time monitoring of Phase 6 production deployment

---

## 📋 Setup Instructions

1. Update `PRODUCTION_URL` below with your actual domain
2. Run cells sequentially (top to bottom)
3. Monitor metrics continuously
4. Check success criteria at the end

In [ ]:
# Configuration
import requests
import json
import time
from datetime import datetime
from typing import Dict, Any, Optional
import warnings
warnings.filterwarnings('ignore')

# ⚙️ UPDATE THIS WITH YOUR PRODUCTION URL
PRODUCTION_URL = "https://your-production-domain.vercel.app"  # Change this!
LOCALHOST_URL = "http://localhost:3000"

# Use localhost if production not available yet
BASE_URL = PRODUCTION_URL if PRODUCTION_URL != "https://your-production-domain.vercel.app" else LOCALHOST_URL

print(f"✅ Configured to monitor: {BASE_URL}")
print(f"📍 Monitoring started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1️⃣ Health Check - Is the API Responding?

In [ ]:
def check_health() -> Dict[str, Any]:
    """Check if API is responding"""
    try:
        response = requests.get(f"{BASE_URL}/api/health", timeout=10, verify=False)
        status = "✅ HEALTHY" if response.status_code == 200 else "❌ ERROR"
        
        return {
            "status": status,
            "http_code": response.status_code,
            "timestamp": datetime.now().isoformat(),
            "response_time_ms": round(response.elapsed.total_seconds() * 1000)
        }
    except Exception as e:
        return {
            "status": "❌ API NOT RESPONDING",
            "error": str(e),
            "timestamp": datetime.now().isoformat()
        }

# Run health check
health = check_health()
print(f"Health Status: {health['status']}")
if 'http_code' in health:
    print(f"HTTP Code: {health['http_code']}")
    print(f"Response Time: {health['response_time_ms']}ms")
else:
    print(f"Error: {health.get('error', 'Unknown')}")
print()

## 2️⃣ Deployment Status - Is Production Ready?

In [ ]:
def check_deployment_status() -> Dict[str, Any]:
    """Check production deployment readiness"""
    try:
        response = requests.get(f"{BASE_URL}/api/deployment/final-report", timeout=10, verify=False)
        if response.status_code == 200:
            data = response.json()
            return {
                "status": data.get('status', 'Unknown'),
                "all_phases_complete": data.get('allPhasesCompleted', False),
                "production_ready": data.get('productionReady', False),
                "estimated_deploy_time": data.get('estimatedProductionDeployment', 'Unknown'),
                "http_code": response.status_code
            }
        else:
            return {"status": "Error", "http_code": response.status_code}
    except Exception as e:
        return {"status": f"Error: {str(e)}"}

# Run deployment status check
deployment = check_deployment_status()
print(f"🚀 Deployment Status: {deployment.get('status', 'Unknown')}")
print(f"✅ All Phases Complete: {deployment.get('all_phases_complete', False)}")
print(f"✅ Production Ready: {deployment.get('production_ready', False)}")
print(f"⏱️  Estimated Deploy Time: {deployment.get('estimated_deploy_time', 'N/A')}")
print()

## 3️⃣ Real-Time Metrics - Monitor Performance

In [ ]:
def get_metrics() -> Dict[str, Any]:
    """Get real-time metrics from dashboard"""
    try:
        response = requests.get(f"{BASE_URL}/api/monitoring/metrics-dashboard", timeout=10, verify=False)
        if response.status_code == 200:
            data = response.json()
            return {
                "success_rate": data.get('successRate', 0),
                "error_rate": data.get('errorRate', 0),
                "p99_latency": data.get('p99Latency', 0),
                "p95_latency": data.get('p95Latency', 0),
                "cache_hit_rate": data.get('cacheHitRate', 0),
                "total_events": data.get('totalEvents', 0),
                "duplicate_rate": data.get('duplicateRate', 0),
                "timestamp": datetime.now().isoformat()
            }
    except Exception as e:
        return {"error": str(e)}

# Get metrics
metrics = get_metrics()

if 'error' not in metrics:
    print("📊 REAL-TIME METRICS:")
    print(f"✅ Success Rate: {metrics['success_rate']:.2f}%", end="")
    print(" ✅" if metrics['success_rate'] > 95 else " ⚠️" if metrics['success_rate'] > 90 else " ❌")
    
    print(f"❌ Error Rate: {metrics['error_rate']:.2f}%", end="")
    print(" ✅" if metrics['error_rate'] < 5 else " ⚠️" if metrics['error_rate'] < 10 else " ❌")
    
    print(f"⏱️  P99 Latency: {metrics['p99_latency']:.0f}ms", end="")
    print(" ✅" if metrics['p99_latency'] < 3000 else " ⚠️" if metrics['p99_latency'] < 5000 else " ❌")
    
    print(f"⏱️  P95 Latency: {metrics['p95_latency']:.0f}ms")
    
    print(f"💾 Cache Hit Rate: {metrics['cache_hit_rate']:.2f}%", end="")
    print(" ✅" if metrics['cache_hit_rate'] > 50 else " ⚠️")
    
    print(f"📈 Total Events: {metrics['total_events']}")
    print(f"🔄 Duplicate Rate: {metrics['duplicate_rate']:.2f}%")
else:
    print(f"⚠️  Metrics not available yet: {metrics['error']}")
print()

## 4️⃣ Sentry Release Health - Error Tracking

In [ ]:
def check_sentry_health() -> Dict[str, Any]:
    """Check Sentry Release Health status"""
    try:
        response = requests.get(f"{BASE_URL}/api/monitoring/release-health", timeout=10, verify=False)
        if response.status_code == 200:
            return response.json()
    except Exception as e:
        return {"error": str(e)}

# Check Sentry health
sentry = check_sentry_health()

print("🔍 SENTRY RELEASE HEALTH:")
if 'error' not in sentry:
    print(f"✅ Status: {sentry.get('status', 'Unknown')}")
    print(f"📊 Session Count: {sentry.get('sessionCount', 0)}")
    print(f"💥 Crash Rate: {sentry.get('crashRate', 0):.2f}%", end="")
    print(" ✅" if sentry.get('crashRate', 100) == 0 else " ❌")
    print(f"🚨 Error Count: {sentry.get('errorCount', 0)}")
else:
    print(f"⚠️  Sentry health not available: {sentry['error']}")
print()

## 5️⃣ Smoke Tests - Validate Functionality

In [ ]:
def run_smoke_tests() -> Dict[str, Dict[str, Any]]:
    """Run smoke tests on critical endpoints"""
    results = {}
    
    # Test 1: Health endpoint
    try:
        r = requests.get(f"{BASE_URL}/api/health", timeout=5, verify=False)
        results['health'] = {"pass": r.status_code == 200, "code": r.status_code}
    except:
        results['health'] = {"pass": False, "error": "timeout"}
    
    # Test 2: Webhook validation
    try:
        payload = {
            "channel": "EMAIL",
            "sender": {"email": "test@example.com"},
            "body": "Test message"
        }
        r = requests.post(
            f"{BASE_URL}/api/webhooks/test-multichannel/phase4",
            json=payload,
            timeout=5,
            verify=False
        )
        results['webhook'] = {"pass": r.status_code == 200, "code": r.status_code}
    except:
        results['webhook'] = {"pass": False, "error": "timeout"}
    
    # Test 3: Validation endpoint
    try:
        r = requests.post(
            f"{BASE_URL}/api/test/webhook-validation",
            json={"channel": "EMAIL", "sender": {"email": "test@example.com"}, "body": "test"},
            timeout=5,
            verify=False
        )
        results['validation'] = {"pass": r.status_code in [200, 400], "code": r.status_code}
    except:
        results['validation'] = {"pass": False, "error": "timeout"}
    
    # Test 4: Metrics endpoint
    try:
        r = requests.get(f"{BASE_URL}/api/monitoring/metrics-dashboard", timeout=5, verify=False)
        results['metrics'] = {"pass": r.status_code == 200, "code": r.status_code}
    except:
        results['metrics'] = {"pass": False, "error": "timeout"}
    
    return results

# Run smoke tests
print("🧪 SMOKE TESTS:")
tests = run_smoke_tests()

for test_name, result in tests.items():
    status = "✅ PASS" if result['pass'] else "❌ FAIL"
    print(f"{status} - {test_name.upper()}", end="")
    if 'code' in result:
        print(f" (HTTP {result['code']})")
    else:
        print(f" ({result.get('error', 'error')})")

passed = sum(1 for r in tests.values() if r.get('pass', False))
total = len(tests)
print(f"\n📊 Tests Passed: {passed}/{total}")
print()

## 6️⃣ Success Criteria Validation

In [ ]:
def validate_success_criteria() -> Dict[str, bool]:
    """Check if deployment meets success criteria"""
    criteria = {}
    
    # Health check
    health = check_health()
    criteria['api_responding'] = health.get('status') == '✅ HEALTHY'
    
    # Deployment status
    deployment = check_deployment_status()
    criteria['production_ready'] = deployment.get('production_ready', False)
    
    # Metrics
    metrics = get_metrics()
    criteria['success_rate_good'] = metrics.get('success_rate', 0) > 95
    criteria['error_rate_low'] = metrics.get('error_rate', 100) < 10
    criteria['latency_acceptable'] = metrics.get('p99_latency', 10000) < 5000
    
    # Sentry health  
    sentry = check_sentry_health()
    criteria['sentry_healthy'] = sentry.get('status') == 'Healthy' if 'status' in sentry else False
    
    # Smoke tests
    tests = run_smoke_tests()
    criteria['smoke_tests_pass'] = sum(1 for r in tests.values() if r.get('pass', False)) >= 3
    
    return criteria

print("✅ SUCCESS CRITERIA VALIDATION:")
criteria = validate_success_criteria()

for criterion, passed in criteria.items():
    status = "✅" if passed else "❌"
    print(f"{status} {criterion.replace('_', ' ').title()}")

total_passed = sum(1 for v in criteria.values() if v)
total_criteria = len(criteria)

print(f"\n📊 Criteria Met: {total_passed}/{total_criteria}")

if total_passed == total_criteria:
    print("\n🎉 ALL SUCCESS CRITERIA MET - DEPLOYMENT SUCCESSFUL!")
elif total_passed >= total_criteria - 1:
    print(f"\n⚠️  ALMOST THERE - {total_criteria - total_passed} criteria pending")
else:
    print(f"\n❌ {total_criteria - total_passed} criteria not met yet - monitoring...")

print()

## 7️⃣ Continuous Monitoring Loop

In [ ]:
def monitoring_loop(iterations: int = 12, interval: int = 30):
        """Run continuous monitoring loop
        
        Args:
            iterations: How many times to check (default 12 = 6 minutes at 30sec intervals)
            interval: Seconds between checks (default 30)
        """
        print(f"🔄 Starting monitoring loop - {iterations} iterations every {interval}s ({iterations * interval / 60:.1f} min total)")
        print("Press Ctrl+C to stop\n")
        
        try:
            for i in range(iterations):
                print(f"\n{'='*60}")
                print(f"Check #{i+1}/{iterations} - {datetime.now().strftime('%H:%M:%S')}")
                print(f"{'='*60}")
                
                metrics = get_metrics()
                if 'error' not in metrics:
                    print(f"✅ Success Rate: {metrics['success_rate']:.2f}%")
                    print(f"❌ Error Rate: {metrics['error_rate']:.2f}%")
                    print(f"⏱️  P99 Latency: {metrics['p99_latency']:.0f}ms")
                    print(f"💾 Cache Hit: {metrics['cache_hit_rate']:.2f}%")
                    print(f"📈 Events: {metrics['total_events']}")
                else:
                    print(f"⚠️  Metrics unavailable: {metrics['error']}")
                
                if i < iterations - 1:
                    print(f"⏳ Next check in {interval}s...", end="")
                    time.sleep(interval)
        except KeyboardInterrupt:
            print("\n\n⏹️  Monitoring stopped by user")

# Ready to run - uncomment to execute continuous monitoring
print("📌 To run continuous monitoring, uncomment and run:")
print("# monitoring_loop(iterations=12, interval=30)")
print("\nOr modify parameters:")
print("# monitoring_loop(iterations=60, interval=10) # 10 minutes at 10-second intervals")

## 📋 Final Deployment Checklist

In [ ]:
print("""\n🎯 DEPLOYMENT CHECKLIST:

BEFORE PRODUCTION:
☐ Update PRODUCTION_URL variable at top of notebook
☐ Verify environment variables configured on platform
☐ Confirm database migrations applied
☐ Enable Sentry Release Health

AFTER MERGE TO MAIN:
☐ Monitor auto-deploy on Vercel/Render dashboard
☐ Wait for 'Building...' → 'Ready' status (5-10 min)
☐ Run Health Check cell above
☐ Run Smoke Tests cell
☐ Monitor metrics for 5-10 minutes
☐ Check success criteria

SUCCESS TARGETS:
✅ Success Rate: > 98% (accept > 95% first 30 min)
✅ Error Rate: < 2% (accept < 5% first 30 min)
✅ P99 Latency: < 3000ms
✅ All Smoke Tests: Pass
✅ Sentry Health: Healthy
✅ No Critical Errors: 0

ROLLBACK TRIGGERS (AUTOMATIC):
❌ Error rate > 5% for 5+ minutes
❌ P99 latency > 5000ms for 5+ minutes
❌ Database connection failures > 20%

RESOURCES:
📊 Sentry: https://sentry.io/organizations/memolib/releases/
🔗 Vercel: https://vercel.com/dashboard
🔗 Render: https://dashboard.render.com
📖 Deployment Guide: DEPLOYMENT_EXECUTION_CHECKLIST.md
📖 Monitoring Guide: PRODUCTION_MONITORING_GUIDE.md
""")

---

**Notebook Version**: 1.0  
**Created**: 2026-02-06  
**Status**: ✅ Production Ready  
**Next Steps**: Update PRODUCTION_URL → Run all cells → Monitor continuously